In [423]:
import requests
import json
import os

In [424]:

jsondir = "tests"

test_filename = os.path.join(os.path.dirname(os.getcwd()), jsondir, 'MONDO0004979Drugfilterred.json')

# Open the file and load the JSON data
with open(test_filename, 'r') as tf:
    answerset = json.load(tf)

answerset["message"].keys()

dict_keys(['query_graph', 'knowledge_graph', 'results', 'auxiliary_graphs'])

In [425]:
query_graph = answerset["message"]["query_graph"]
kg_edges = answerset["message"]["knowledge_graph"]["edges"]
kg_nodes = answerset["message"]["knowledge_graph"]["nodes"]
results = answerset["message"]["results"]
aux_graphs = answerset["message"]["auxiliary_graphs"]

for qedge_id, qedges  in query_graph.get("edges", {}).items():
    subject = query_graph.get("nodes", {})[qedges["subject"]]
    object = query_graph.get("nodes", {})[qedges["object"]]
    if subject.get("ids", []):
        is_source = True
    else:
        is_source = False


# Let's trace one result

In [426]:
len(results)

1801

In [427]:
support_graphs = None
for i in range(len(results)):
    for eid, edge in results[i]["analyses"][0]["edge_bindings"].items():
        inferred_edge = edge[0]["id"]
        inferred_edge_string = f'{kg_nodes[kg_edges[inferred_edge]["subject"]]["name"]}-({kg_edges[inferred_edge]["predicate"]})->{kg_nodes[kg_edges[inferred_edge]["object"]]["name"]}'
        #Let's see the inferred_edge in the KG then extract it support graphs
        support_graphs = [attributes["value"] for attributes in kg_edges[inferred_edge]["attributes"] if attributes["attribute_type_id"] == "biolink:support_graphs"]
        # if len(support_graphs)==1: 
        print(f'{i+1}: {inferred_edge_string}')
                # print(i)
                # break


1: 9-fluoro-3-methyl-10-(4-methylpiperazin-1-yl)-7-oxo-2,3-dihydro-7H-[1,4]oxazino[2,3,4-ij]quinoline-6-carboxylic acid-(biolink:treats)->asthma
2: Revefenacin-(biolink:treats)->asthma
3: olodaterol-(biolink:treats)->asthma
4: aclidinium-(biolink:treats)->asthma
5: roflumilast-(biolink:treats)->asthma
6: 2-cyclopentyl-2-hydroxy-2-phenylacetic acid (1,1-dimethyl-3-pyrrolidin-1-iumyl) ester-(biolink:treats)->asthma
7: Doxycycline-(biolink:treats)->asthma
8: ciprofloxacin-(biolink:treats)->asthma
9: N-acetyl-L-cysteine-(biolink:treats)->asthma
10: levofloxacin-(biolink:treats)->asthma
11: moxifloxacin-(biolink:treats)->asthma
12: desloratadine-(biolink:treats)->asthma
13: clofedanol-(biolink:treats)->asthma
14: prostaglandin I2-(biolink:treats)->asthma
15: hydroxyzine-(biolink:treats)->asthma
16: cyclizine-(biolink:treats)->asthma
17: brompheniramine-(biolink:treats)->asthma
18: astemizole-(biolink:treats)->asthma
19: 1-phenyl-1-cyclopentanecarboxylic acid 2-[2-(diethylamino)ethoxy]ethyl 

In [428]:
support_graphs = None
# for i in range(len(results)):
for eid, edge in results[4]["analyses"][0]["edge_bindings"].items():
    inferred_edge = edge[0]["id"]
    #Let's see the inferred_edge in the KG then extract it support graphs
    support_graphs = [attributes["value"] for attributes in kg_edges[inferred_edge]["attributes"] if attributes["attribute_type_id"] == "biolink:support_graphs"]
    support_graphs = sorted(support_graphs)
    # if len(support_graphs) > 1: 
    print(f'{inferred_edge}==> \n{" ^ \n".join(support_graphs)}')
        # print(i)
        # break


CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979==> 
e_Inferred_SG:_CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979_via_CHEBI:47657_biolink:treats_MONDO:0005002 ^ 
e_Inferred_SG:_CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979_via_CHEBI:47657_biolink:treats_UMLS:C0730607 ^ 
n_Inferred_SG:_CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979_via_CHEBI:47657_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_agent ^ 
n_Inferred_SG:_CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979_via_CHEBI:47657_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug


In [429]:
g_support_graph = support_graphs[0] # Graph(edge) Inferred
p_support_graph = support_graphs[-1] # Property(node) Inferred

#### This implies that the inference was driven by both graph enrichment and property enrichment

### 1. GRAPH ENRICHMENT ROUTE: e_Inferred_SG:........'


In [430]:
# Tracing the support graph to the auxiliary components
aux_graph_edges = aux_graphs.get(g_support_graph).get("edges")
print(f'{inferred_edge}---> {" ^ ".join(aux_graph_edges)}')

# # The aux_graph has 3 edges
# 1. Inferred_node to the enrichment
# 2. Enrichment to the Groupset enriched
# 3. the Groupset enriched to the qg Curie

CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979---> CHEBI:47657_biolink:treats_MONDO:0005002 ^ e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002 ^ uuid:1_biolink:treats_MONDO:0004979


In [431]:
aux_graph_edges

['CHEBI:47657_biolink:treats_MONDO:0005002',
 'e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
 'uuid:1_biolink:treats_MONDO:0004979']

##### Let's dig further down the path

In [432]:
inf2enrichment_aux_graph_edge = ''
enrich2group_aux_graph_edge = ''
group2curie_aux_graph_edge = ''
for aedge in aux_graph_edges:
    support = [attributes["value"] for attributes in kg_edges[aedge]["attributes"]]
    if support:
        if isinstance(support[0], list):
            enrich2group_aux_graph_edge = aedge
        else:
            group2curie_aux_graph_edge = aedge
    else:
        inf2enrichment_aux_graph_edge = aedge

In [433]:
# The first edge: 
inf2enrichmentedge=kg_edges[inf2enrichment_aux_graph_edge]
inf2enrichmentedge

{'subject': 'CHEBI:47657',
 'object': 'MONDO:0005002',
 'predicate': 'biolink:treats',
 'sources': [{'resource_id': 'infores:hetionet',
   'resource_role': 'primary_knowledge_source'}],
 'attributes': []}

In [434]:
print(f"{kg_nodes[inf2enrichmentedge['subject']]['name']} -({inf2enrichmentedge['predicate']})-> {kg_nodes[inf2enrichmentedge['object']]['name']}")
print()
print(inf2enrichmentedge)
# implies:  inferred_node -(rel)- enrichment. It has no support graph so we stop digging

roflumilast -(biolink:treats)-> chronic obstructive pulmonary disease

{'subject': 'CHEBI:47657', 'object': 'MONDO:0005002', 'predicate': 'biolink:treats', 'sources': [{'resource_id': 'infores:hetionet', 'resource_role': 'primary_knowledge_source'}], 'attributes': []}


In [435]:
# The second edge: 
enrichment2group_edge = kg_edges[enrich2group_aux_graph_edge]
enrichment2group_edge
# This is the enrichment - (rel) - lookup_set 

{'subject': 'uuid:1',
 'object': 'MONDO:0005002',
 'predicate': 'biolink:treats_or_applied_or_studied_to_treat',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'qualifiers': [],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': ['SG:_e_CHEBI:8382_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_CHEBI:75037_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_CHEBI:91495_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_CHEBI:3077_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_PUBCHEM.COMPOUND:9433_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_PUBCHEM.COMPOUND:5487426_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_CHEBI:5147_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002',
    'SG:_e_CHEBI:8378_biolink:treats_or_applied_or_studied_to_treat_MONDO:0

In [436]:
# The enrichment - groupset has many support graph, so we dig further
print(f"{enrich2group_aux_graph_edge}:===>")
print()
for attributes in enrichment2group_edge["attributes"]:
    enrichment2group_support_graphs = attributes["value"]
    # Each of these exists in the auxiliary graph
    for i, e2group_sp in enumerate(enrichment2group_support_graphs):
        e2group_edges = aux_graphs[e2group_sp]["edges"]
        theedges0 = kg_edges[e2group_edges[0]]
        theedges1 = kg_edges[e2group_edges[1]]
        print(f'Path{i+1}: {kg_nodes[theedges0["subject"]]["name"]}--({theedges0["predicate"]})-->{kg_nodes[theedges0["object"]]["name"]} ^ {kg_nodes[theedges1["subject"]]["name"]}--({theedges1["predicate"]})-->{kg_nodes[theedges1["object"]]["name"]}')
        print()
# 4 members of set uuid affects `NCBIGene:83817`         

e_uuid:1_biolink:treats_or_applied_or_studied_to_treat_MONDO:0005002:===>

Path1: prednisone--(biolink:member_of)-->uuid:1 ^ prednisone--(biolink:treats_or_applied_or_studied_to_treat)-->chronic obstructive pulmonary disease

Path2: vilanterol--(biolink:treats_or_applied_or_studied_to_treat)-->chronic obstructive pulmonary disease ^ vilanterol--(biolink:member_of)-->uuid:1

Path3: 3-hydroxy-2-phenylpropanoic acid (8-methyl-8-propan-2-yl-8-azoniabicyclo[3.2.1]octan-3-yl) ester--(biolink:member_of)-->uuid:1 ^ 3-hydroxy-2-phenylpropanoic acid (8-methyl-8-propan-2-yl-8-azoniabicyclo[3.2.1]octan-3-yl) ester--(biolink:treats_or_applied_or_studied_to_treat)-->chronic obstructive pulmonary disease

Path4: betamethasone--(biolink:member_of)-->uuid:1 ^ betamethasone--(biolink:treats_or_applied_or_studied_to_treat)-->chronic obstructive pulmonary disease

Path5: Aminophylline--(biolink:member_of)-->uuid:1 ^ Aminophylline--(biolink:treats_or_applied_or_studied_to_treat)-->chronic obstructive pulmo

In [437]:
# The third edge: 
print(kg_edges[group2curie_aux_graph_edge])

# implies:  the Groupset- qg Curie edge. It has support graph of members of the group, so we stop digging

{'subject': 'uuid:1', 'object': 'MONDO:0004979', 'predicate': 'biolink:treats', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'attributes': [{'attribute_type_id': 'biolink:support_graphs', 'value': 'SG:_uuid:1_biolink:treats_MONDO:0004979', 'attribute_source': 'infores:answercoalesce'}]}


In [438]:
# CURIE-PHYSIO
print(f"{group2curie_aux_graph_edge}:===>")
for attributes in kg_edges[group2curie_aux_graph_edge]["attributes"]:
    sgs = attributes['value']
    sgedges = sorted(aux_graphs[sgs]["edges"])
    j = 0; path = 1
    while j < len(sgedges)-1:
        p1 = kg_edges[sgedges[j]]
        p2 = kg_edges[sgedges[j+1]]
        print(f'Path{path}: {kg_nodes[p2["subject"]]["name"]}-({p2["predicate"]})->{kg_nodes[p2["object"]]["name"]} ^ {kg_nodes[p1["subject"]]["name"]}-({p1["predicate"]})->{kg_nodes[p1["object"]]["name"]}')
        j+=2
        path+=1

uuid:1_biolink:treats_MONDO:0004979:===>
Path1: zileuton-(biolink:member_of)->uuid:1 ^ zafirlukast-(biolink:member_of)->uuid:1
Path2: (-)-ephedrine-(biolink:member_of)->uuid:1 ^ pemirolast-(biolink:member_of)->uuid:1
Path3: desonide-(biolink:member_of)->uuid:1 ^ (R)-noradrenaline-(biolink:member_of)->uuid:1
Path4: theophylline-(biolink:member_of)->uuid:1 ^ albuterol-(biolink:member_of)->uuid:1
Path5: beclomethasone-(biolink:member_of)->uuid:1 ^ (R)-adrenaline-(biolink:member_of)->uuid:1
Path6: Ciclesonide-(biolink:member_of)->uuid:1 ^ betamethasone-(biolink:member_of)->uuid:1
Path7: Cortisone acetate-(biolink:member_of)->uuid:1 ^ budesonide-(biolink:member_of)->uuid:1
Path8: Dexamethasone acetate-(biolink:member_of)->uuid:1 ^ arformoterol-(biolink:member_of)->uuid:1
Path9: dyphylline-(biolink:member_of)->uuid:1 ^ troleandomycin-(biolink:member_of)->uuid:1
Path10: flunisolide-(biolink:member_of)->uuid:1 ^ montelukast-(biolink:member_of)->uuid:1
Path11: fluticasone-(biolink:member_of)->u

In [415]:
# PROPERTY ENRICHED SG
p_support_graph = support_graphs[-1] # Property Inferred

In [416]:
# Tracing the support graph to the auxiliary components
aux_graph_edges = aux_graphs.get(p_support_graph).get("edges")
print(f'{inferred_edge}---> {" ^ ".join(aux_graph_edges)}')

# # The aux_graph has 3 edges
# 1. Inferred_node to the enrichment
# 2. Enrichment to the Groupset enriched
# 3. the Groupset enriched to the qg Curie

CHEBI:47657_Inferred_to_biolink:treats_MONDO:0004979---> e_CHEBI_ROLE_anti_asthmatic_drug_biolink:similar_to_uuid:1 ^ CHEBI:47657_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug ^ uuid:1_biolink:treats_MONDO:0004979


In [417]:
inf2enrichment_aux_graph_edge = ''
enrich2group_aux_graph_edge = ''
group2curie_aux_graph_edge = ''
for aedge in aux_graph_edges:
    support = [attributes["value"] for attributes in kg_edges[aedge]["attributes"]]
    if support:
        if isinstance(support[0], list):
            enrich2group_aux_graph_edge = aedge
        else:
            group2curie_aux_graph_edge = aedge
    else:
        inf2enrichment_aux_graph_edge = aedge

##### Let's dig further down the path

In [418]:
# The first edge: 
inf2enrichmentedges=kg_edges[inf2enrichment_aux_graph_edge]

print(f"{kg_nodes[inf2enrichmentedges['subject']]['name']} -({inf2enrichmentedges['predicate']})-> {kg_nodes[inf2enrichmentedges['object']]['name']}")
print()
print(inf2enrichmentedges)
# implies:  inferred_node -(rel)- enrichment. It has no support graph so we stop digging

roflumilast -(biolink:has_chemical_role)-> CHEBI_ROLE_anti_asthmatic_drug

{'subject': 'CHEBI:47657', 'object': 'CHEBI_ROLE_anti_asthmatic_drug', 'predicate': 'biolink:has_chemical_role', 'sources': [{'resource_id': 'infores:answercoalesce', 'resource_role': 'primary_knowledge_source'}], 'attributes': []}


In [419]:
# The second edge: 
enrichment2group_edge = kg_edges[enrich2group_aux_graph_edge]
enrichment2group_edge
# This is the enrichment - (rel) - lookup_set 

{'subject': 'CHEBI_ROLE_anti_asthmatic_drug',
 'object': 'uuid:1',
 'predicate': 'biolink:similar_to',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'qualifiers': [],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': ['SG:_n_CHEBI:10112_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:7492_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:59773_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:51209_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:50730_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:6792_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:408174_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:9449_biolink:has_chemical_role_CHEBI_ROLE_anti_asthmatic_drug',
    'SG:_n_CHEBI:5106_biolink:has_chemical_rol

In [420]:
# The enrichment - groupset has many support graph, so we dig further
print(f"{enrich2group_aux_graph_edge}:===>")
for attributes in enrichment2group_edge["attributes"]:
    enrichment2group_support_graphs = attributes["value"]
    
    # Each of these exists in the auxiliary graph
    for i, e2group_sp in enumerate(enrichment2group_support_graphs):
        e2group_edges = aux_graphs[e2group_sp]["edges"]
        theedges0 = kg_edges[e2group_edges[0]]
        theedges1 = kg_edges[e2group_edges[1]]
        print(f'Path{i+1}: {kg_nodes[theedges0["subject"]]["name"]}--({theedges0["predicate"]})-->{kg_nodes[theedges0["object"]]["name"]} ^ {kg_nodes[theedges1["subject"]]["name"]}--({theedges1["predicate"]})-->{kg_nodes[theedges1["object"]]["name"]}')

# 4 members of set uuid affects `NCBIGene:83817`         

e_CHEBI_ROLE_anti_asthmatic_drug_biolink:similar_to_uuid:1:===>
Path1: zileuton--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug ^ zileuton--(biolink:member_of)-->uuid:1
Path2: nedocromil--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug ^ nedocromil--(biolink:member_of)-->uuid:1
Path3: cromoglycic acid--(biolink:member_of)-->uuid:1 ^ cromoglycic acid--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug
Path4: pseudoephedrine--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug ^ pseudoephedrine--(biolink:member_of)-->uuid:1
Path5: montelukast--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug ^ montelukast--(biolink:member_of)-->uuid:1
Path6: metaproterenol--(biolink:member_of)-->uuid:1 ^ metaproterenol--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug
Path7: arformoterol--(biolink:member_of)-->uuid:1 ^ arformoterol--(biolink:has_chemical_role)-->CHEBI_ROLE_anti_asthmatic_drug
Path8: terbutaline--(biolink:has_chemical_r

In [421]:
# The third edge: 
kg_edges[group2curie_aux_graph_edge]
# implies:  the Groupset- qg Curie edge. It has support graph of members of the group, so we stop digging

{'subject': 'uuid:1',
 'object': 'MONDO:0004979',
 'predicate': 'biolink:treats',
 'sources': [{'resource_id': 'infores:answercoalesce',
   'resource_role': 'primary_knowledge_source'}],
 'attributes': [{'attribute_type_id': 'biolink:support_graphs',
   'value': 'SG:_uuid:1_biolink:treats_MONDO:0004979',
   'attribute_source': 'infores:answercoalesce'}]}

In [422]:
# CURIE-PHYSIO
print(f"{group2curie_aux_graph_edge}:===>")
for attributes in kg_edges[group2curie_aux_graph_edge]["attributes"]:
    sgs = attributes['value']
    sgedges = sorted(aux_graphs[sgs]["edges"])
    j = 0; path = 1
    while j < len(sgedges)-1:
        p1 = kg_edges[sgedges[j]]
        p2 = kg_edges[sgedges[j+1]]
        print(f'Path{path}: {kg_nodes[p2["subject"]]["name"]}-({p2["predicate"]})->{kg_nodes[p2["object"]]["name"]} ^ {kg_nodes[p1["subject"]]["name"]}-({p1["predicate"]})->{kg_nodes[p1["object"]]["name"]}')
        j+=2
        path+=1

uuid:1_biolink:treats_MONDO:0004979:===>
Path1: zafirlukast-(biolink:member_of)->uuid:1 ^ zafirlukast-(biolink:treats)->asthma
Path2: zileuton-(biolink:member_of)->uuid:1 ^ zileuton-(biolink:treats)->asthma
Path3: pemirolast-(biolink:member_of)->uuid:1 ^ pemirolast-(biolink:treats)->asthma
Path4: (-)-ephedrine-(biolink:member_of)->uuid:1 ^ (-)-ephedrine-(biolink:treats)->asthma
Path5: (R)-noradrenaline-(biolink:member_of)->uuid:1 ^ (R)-noradrenaline-(biolink:treats)->asthma
Path6: desonide-(biolink:member_of)->uuid:1 ^ desonide-(biolink:treats)->asthma
Path7: albuterol-(biolink:member_of)->uuid:1 ^ albuterol-(biolink:treats)->asthma
Path8: theophylline-(biolink:member_of)->uuid:1 ^ theophylline-(biolink:treats)->asthma
Path9: (R)-adrenaline-(biolink:member_of)->uuid:1 ^ (R)-adrenaline-(biolink:treats)->asthma
Path10: beclomethasone-(biolink:member_of)->uuid:1 ^ beclomethasone-(biolink:treats)->asthma
Path11: betamethasone-(biolink:member_of)->uuid:1 ^ betamethasone-(biolink:treats)->as